## how to train NER from scratch using Bert with Pytorch

In [10]:
import pandas as pd
df=pd.read_csv('sv_data/train_no_nulls.tsv',names=['words','token'], sep='\t')
df['token'].unique() 

array(['O', 'PER', 'ORG', 'LOC', 'MISC', 'PRG'], dtype=object)

In [ ]:
import pandas as pd
def readfile(filename):
    
    df=pd.read_csv(filename,names=['words','token'], sep='\t')
    n=len(df.index)

    data=[]
    sents=[]
    labels=[]
    for i in range(n):
        if df.iloc[i,0] not in ['.',',','-','...','!','?','¡']:
            word=df.iloc[i,0]
            label=df.iloc[i,1]
            #print(word, label)
            sents.append(word)
            labels.append(label)
        elif df.iloc[i,0] in ['.',',','-','...','!','?','¡']:
            #print(sents,labels)
            word=df.iloc[i,0]
            label=df.iloc[i,1]
            sents.append(word)
            labels.append(label)
            data.append((sents,labels))
            sents=[]
            labels=[]
        else :
            print("something is strange ", df.iloc[i,0])
    return data

data=readfile('sv_data/train_no_nulls.tsv')

In [ ]:
max_len=0
n=len(data)
i=0
for (s,t) in data:  
    if i<=n:
        print(len(s),len(t),'\n')
        print(s,t,'\n')
        i+=1
    else:break
print(max_len)

In [1]:
!python run_ner_sv.py --data_dir=sv_data/train_no_nulls.tsv --bert_model=bert-base-cased  --task_name=ner --output_dir=out_ner_sv --max_seq_length=65 --do_train --num_train_epochs 5 --do_eval --warmup_proportion=0.1

09/16/2020 11:03:33 - INFO - __main__ -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
09/16/2020 11:03:34 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache, downloading to /tmp/tmp27x03i0c
100%|████████████████████████████████| 213450/213450 [00:00<00:00, 555038.42B/s]
09/16/2020 11:03:35 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmp27x03i0c to cache at /root/.cache/torch/pytorch_transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
09/16/2020 11:03:35 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
09/16/2020 11:03:35 - INFO - pytorch_transformers.file_utils -   removin

In [2]:
%%writefile bert.py
"""BERT NER Inference."""

from __future__ import absolute_import, division, print_function

import json
import os

import torch
import torch.nn.functional as F
from nltk import word_tokenize
from pytorch_transformers import (BertConfig, BertForTokenClassification,
                                  BertTokenizer)


class BertNer(BertForTokenClassification):

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, valid_ids=None):
        sequence_output = self.bert(input_ids, token_type_ids, attention_mask, head_mask=None)[0]
        batch_size,max_len,feat_dim = sequence_output.shape
        valid_output = torch.zeros(batch_size,max_len,feat_dim,dtype=torch.float32,device='cuda' if torch.cuda.is_available() else 'cpu')
        for i in range(batch_size):
            jj = -1
            for j in range(max_len):
                    if valid_ids[i][j].item() == 1:
                        jj += 1
                        valid_output[i][jj] = sequence_output[i][j]
        sequence_output = self.dropout(valid_output)
        logits = self.classifier(sequence_output)
        return logits

class Ner:

    def __init__(self,model_dir: str):
        self.model , self.tokenizer, self.model_config = self.load_model(model_dir)
        self.label_map = self.model_config["label_map"]
        self.max_seq_length = self.model_config["max_seq_length"]
        self.label_map = {int(k):v for k,v in self.label_map.items()}
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = self.model.to(self.device)
        self.model.eval()

    def load_model(self, model_dir: str, model_config: str = "model_config.json"):
        model_config = os.path.join(model_dir,model_config)
        model_config = json.load(open(model_config))
        model = BertNer.from_pretrained(model_dir)
        tokenizer = BertTokenizer.from_pretrained(model_dir, do_lower_case=model_config["do_lower"])
        return model, tokenizer, model_config

    def tokenize(self, text: str):
        """ tokenize input"""
        words = word_tokenize(text)
        tokens = []
        valid_positions = []
        for i,word in enumerate(words):
            token = self.tokenizer.tokenize(word)
            tokens.extend(token)
            for i in range(len(token)):
                if i == 0:
                    valid_positions.append(1)
                else:
                    valid_positions.append(0)
        return tokens, valid_positions

    def preprocess(self, text: str):
        """ preprocess """
        tokens, valid_positions = self.tokenize(text)
        ## insert "[CLS]"
        tokens.insert(0,"[CLS]")
        valid_positions.insert(0,1)
        ## insert "[SEP]"
        tokens.append("[SEP]")
        valid_positions.append(1)
        segment_ids = []
        for i in range(len(tokens)):
            segment_ids.append(0)
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)
        while len(input_ids) < self.max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)
            valid_positions.append(0)
        return input_ids,input_mask,segment_ids,valid_positions

    def predict(self, text: str):
        input_ids,input_mask,segment_ids,valid_ids = self.preprocess(text)
        input_ids = torch.tensor([input_ids],dtype=torch.long,device=self.device)
        input_mask = torch.tensor([input_mask],dtype=torch.long,device=self.device)
        segment_ids = torch.tensor([segment_ids],dtype=torch.long,device=self.device)
        valid_ids = torch.tensor([valid_ids],dtype=torch.long,device=self.device)
        with torch.no_grad():
            logits = self.model(input_ids, segment_ids, input_mask,valid_ids)
        logits = F.softmax(logits,dim=2)
        logits_label = torch.argmax(logits,dim=2)
        logits_label = logits_label.detach().cpu().numpy().tolist()[0]

        logits_confidence = [values[label].item() for values,label in zip(logits[0],logits_label)]

        logits = []
        pos = 0
        for index,mask in enumerate(valid_ids[0]):
            if index == 0:
                continue
            if mask == 1:
                logits.append((logits_label[index-pos],logits_confidence[index-pos]))
            else:
                pos += 1
        logits.pop()

        labels = [(self.label_map[label],confidence) for label,confidence in logits]
        words = word_tokenize(text)
        assert len(labels) == len(words)

        Person = []
        Location = []
        Organization = []
        Miscelleneous = []
        
        for word, (label, confidence) in zip(words, labels):
            if label=="PER" :
                Person.append(word)
            elif label=="LOC" :
                Location.append(word)
            elif label=="ORG" :
                Organization.append(word)
            elif label=="MISC" :
                Miscelleneous.append(word)
            else:
                output = None

        output = []
        for word, (label, confidence) in zip(words, labels):      
            if label == "PER":
                output.append(' '.join(Person) + ": Person")
            if label=="LOC":
                output.append(' '.join(Location) + ": Location")
            if label=="MISC":
                output.append(' '.join(Miscelleneous) + ": Miscelleneous Entity")
            if label=="ORG":
                output.append(' '.join(Organization) + ": Organization")
                
        return output

Overwriting bert.py


In [2]:
# we need nltk to handle punctuations  
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# make prediction 
from bert import Ner

model = Ner("out_ner_sv/")

output = model.predict("Heidi Åkesson jobbar på IKEA")

print(output)

['Heidi Åkesson: Person', 'Heidi Åkesson: Person', 'IKEA: Organization']


In [4]:
output = model.predict("Det finns många roliga aktiviteter i Göteborg")

print(output)

['Göteborg: Location']


In [5]:
output = model.predict("Henrik studerar i Uppsala Universitet")

print(output)

['Henrik: Person', 'Uppsala Universitet: Organization', 'Uppsala Universitet: Organization']
